In [1]:
# Step 1: Import the necessary libraries
import pandas as pd
import statsmodels.api as sm
from pandas_datareader import data as pdr

In [2]:
# Step 2: Load the data
file_path = './data/spus_2024.csv'
strategies_prices = pd.read_csv(file_path, parse_dates=True, index_col=0)

/tmp/ipykernel_100192/2726091380.py:3: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  strategies_prices = pd.read_csv(file_path, parse_dates=True, index_col=0)


In [3]:
# Step 3: Separate SPX and strategies
universe_name = 'SPUS Bench'
universe_prices = strategies_prices[universe_name]
risk_premia_spus_prices = strategies_prices.drop(columns=[universe_name])

# Convert the strategies to returns
strategies = risk_premia_spus_prices.pct_change().dropna()
universe = universe_prices.pct_change().dropna()

In [4]:
# Step 4: Retrieve the Fama-French 4-factor data
ff_factors = pdr.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start='2024-01-01', end='2024-04-30')[0]
momentum_factor = pdr.DataReader('F-F_Momentum_Factor_daily', 'famafrench', start='2024-01-01', end='2024-04-30')[0]

# Combine the Fama-French factors and the momentum factor into a single DataFrame
ff_factors = ff_factors.join(momentum_factor['Mom   '])

# Convert the factors to daily returns as percentages
ff_factors = ff_factors / 100

/tmp/ipykernel_100192/3849217727.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff_factors = pdr.DataReader('F-F_Research_Data_Factors_daily', 'famafrench', start='2024-01-01', end='2024-04-30')[0]
/tmp/ipykernel_100192/3849217727.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  momentum_factor = pdr.DataReader('F-F_Momentum_Factor_daily', 'famafrench', start='2024-01-01', end='2024-04-30')[0]


In [5]:
# Step 5: Run the Fama-French 4-factor regression for each strategy
model_results = {}
for strategy_name in strategies.columns:
    # Merge strategy returns with the Fama-French factors
    data = pd.concat([strategies[strategy_name], ff_factors], axis=1, join='inner')
    
    # Prepare the data for regression
    X = data[['Mkt-RF', 'SMB', 'HML', 'Mom   ']]
    y = data[strategy_name] - data['RF']
    
    # Add a constant to the independent variables
    X = sm.add_constant(X)
    
    # Run the regression
    model = sm.OLS(y, X).fit()
    
    # Store the model results
    model_results[strategy_name] = model

In [6]:
# Extract the coefficients and p-values for each strategy into 2 distinctive tables
coefficients = pd.DataFrame({strategy: model_results[strategy].params for strategy in model_results})
p_values = pd.DataFrame({strategy: model_results[strategy].pvalues for strategy in model_results})

# Round the values to 3 decimal places
coefficients = coefficients.round(3)
p_values = p_values.round(3)

# Step 6: Display the results
print('Coefficients:')
print(coefficients)
print('\nP-Values:')
print(p_values)

Coefficients:
        SPUS Value  SPUS Momentum  SPUS Quality  SPUS Low Beta  SPUS Ptf
const       -0.000         -0.001         0.000          0.000     0.000
Mkt-RF       1.126          1.087         1.001          0.823     0.989
SMB         -0.025          0.093        -0.004         -0.168    -0.004
HML          0.289         -0.005        -0.036         -0.093     0.076
Mom         -0.228          0.502         0.043         -0.313     0.023

P-Values:
        SPUS Value  SPUS Momentum  SPUS Quality  SPUS Low Beta  SPUS Ptf
const        0.972          0.315         0.511          0.558     0.838
Mkt-RF       0.000          0.000         0.000          0.000     0.000
SMB          0.681          0.183         0.938          0.002     0.901
HML          0.001          0.961         0.595          0.193     0.117
Mom          0.003          0.000         0.464          0.000     0.588


In [7]:
# Add the universe results to these two tables
universe_data = pd.concat([universe, ff_factors], axis=1, join='inner')
X = universe_data[['Mkt-RF', 'SMB', 'HML', 'Mom   ']]
y = universe_data[universe_name] - universe_data['RF']
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
universe_coefficients = model.params.round(3)
universe_p_values = model.pvalues.round(3)

print('\nUniverse Coefficients:')
print(universe_coefficients)
print('\nUniverse P-Values:')
print(universe_p_values)



Universe Coefficients:
const     0.000
Mkt-RF    1.009
SMB      -0.068
HML      -0.007
Mom       0.020
dtype: float64

Universe P-Values:
const     0.979
Mkt-RF    0.000
SMB       0.000
HML       0.789
Mom       0.363
dtype: float64


In [8]:
# Merge universe resuts with the strategy results without append method 
coefficients = pd.concat([coefficients, universe_coefficients], axis=1)
p_values = pd.concat([p_values, universe_p_values], axis=1)
# Add universe name as field name
coefficients.columns = coefficients.columns.tolist()[:-1] + [universe_name]
p_values.columns = p_values.columns.tolist()[:-1] + [universe_name]


# Display the updated tables with no wrap because of too many rows
pd.set_option('display.expand_frame_repr', False)
print('\nUpdated Coefficients:')
print(coefficients)
print('\nUpdated P-Values:')
print(p_values)

# Reset the display settings
pd.reset_option('display.expand_frame_repr')


Updated Coefficients:
        SPUS Value  SPUS Momentum  SPUS Quality  SPUS Low Beta  SPUS Ptf  SPUS Bench
const       -0.000         -0.001         0.000          0.000     0.000       0.000
Mkt-RF       1.126          1.087         1.001          0.823     0.989       1.009
SMB         -0.025          0.093        -0.004         -0.168    -0.004      -0.068
HML          0.289         -0.005        -0.036         -0.093     0.076      -0.007
Mom         -0.228          0.502         0.043         -0.313     0.023       0.020

Updated P-Values:
        SPUS Value  SPUS Momentum  SPUS Quality  SPUS Low Beta  SPUS Ptf  SPUS Bench
const        0.972          0.315         0.511          0.558     0.838       0.979
Mkt-RF       0.000          0.000         0.000          0.000     0.000       0.000
SMB          0.681          0.183         0.938          0.002     0.901       0.000
HML          0.001          0.961         0.595          0.193     0.117       0.789
Mom          0.003     

In [12]:
# Step 7: Save the results to a CSV file
coefficients.to_csv('./data/coefficients.csv')
p_values.to_csv('./data/p_values.csv')

In [ ]:
```
The code above is a Python script that performs the following steps:

1. Import the necessary libraries.
2. Load the data from the CSV file.
3. Separate the S&P 500 index and the strategies, convert the prices to returns, and calculate the Fama-French factors and the momentum factor.
4. Run the Fama-French 4-factor regression for each strategy and store the model results.
5. Extract the coefficients and p-values for each strategy and the S&P 500 index into two separate tables.
6. Display the results and add the S&P 500 index results to the tables.
7. Save the results to CSV files.

The script uses the `pandas`, `statsmodels`, and `pandas_datareader` libraries to load and analyze the data. It calculates the Fama-French factors and momentum factor, runs the regression analysis, and outputs the coefficients and p-values for each strategy and the S&P 500 index.

You can run this script in a Python environment to analyze the Fama-French 4-factor model for the strategies and the S&P 500 index. The results will be saved to CSV files for further analysis or reporting.

Please note that you may need to install the required libraries (`pandas`, `statsmodels`, `pandas_datareader`) if you haven't already done so. You can install them using `pip` by running the following commands in your Python environment:

```bash
